In [2]:
#os.environ['OPENAI_API_KEY'] = <your-api-key>
import openai
import os

api_key = os.getenv('OPENAI_API_KEY')

https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

In [13]:

import pandas as pd
import re
def data_loader():
  """Loads data from CSV files, performs data cleaning and feature engineering.

  Returns:
    pandas.DataFrame: The processed hotel reviews dataset.
  """

  # Read hotel reviews data
  Hotel_Reviews = pd.read_csv("../Data/Hotel_Reviews.csv")

  # Read country data
  countries = pd.read_csv("../Data/countries.csv")

  # Add a "Country" column with NA values
  Hotel_Reviews["Country"] = pd.NA

  # Assign country based on pattern matching in Hotel_Address
  for country_name in countries["name"]:
    pattern = re.compile(country_name, re.IGNORECASE)  # Case-insensitive matching
    Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name

 
  # Convert Review_Date to datetime and extract features
  Hotel_Reviews["date_object"] = pd.to_datetime(Hotel_Reviews["Review_Date"], format="%m/%d/%Y")
  #Hotel_Reviews["time"] = Hotel_Reviews["date_object"].astype(int) / 10**9  # Convert to Unix timestamp
  Hotel_Reviews["month"] = Hotel_Reviews["date_object"].dt.month
  Hotel_Reviews["num_date_object"] = Hotel_Reviews["date_object"].dt.day_of_year / 365  # Normalize by days in a year

  return Hotel_Reviews

# Load the processed data
Hotel_Reviews = data_loader()

C:\Users\alan_\AppData\Local\Temp\ipykernel_42836\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_42836\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_42836\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_42836\278692378.py:22: UserWarning: This pattern is inte

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter



# Merge the columns using string concatenation
Hotel_Reviews['MergedColumn'] = (
     '' +'Hotel: ' + Hotel_Reviews['Hotel_Name'] + 
    '. Guest Review: ' + Hotel_Reviews['Positive_Review'] + 
    '. ' + Hotel_Reviews['Negative_Review'] + "\n"
)
# Select the first 100 rows of the merged column
used_data = Hotel_Reviews['MergedColumn'][:200]

# Specify the file name
file_name = "used_data.txt"

# Save the data to a text file
with open(file_name, 'w') as f:
    for line in used_data:
        f.write(line + '\n')

    


#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [4]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   -- ------------------------------------- 1.3/24.0 MB 9.6 MB/s eta 0:00:03
   ----- ---------------------------------- 3.4/24.0 MB 11.2 MB/s eta 0:00:02
   -------- ------------------------------- 5.0/24.0 MB 9.2 MB/s eta 0:00:03
   ------------- -------------------------- 7.9/24.0 MB 10.8 MB/s eta 0:00:02
   ----------------- ---------------------- 10.2/24.0 MB 10.6 MB/s eta 0:00:02
   --------------------- ------------------ 12.8/24.0 MB 11.8 MB/s eta 0:00:01
   --------------------------- ------------ 16.3/24.0 MB 12.0 MB/s eta 0:00:01
   --------------------------------- ------ 19.9/24.0 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------  23.6/24.0 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   --- ------------------------------------ 3.7/46.2 MB 18.1 MB/s eta 0:00:0

In [ ]:
import nltk
nltk.download()

In [10]:
from nltk.stem import SnowballStemmer
import gensim
from nltk.stem import WordNetLemmatizer

'''
Write a function to proprocess the entire dataset 
'''
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    '''This function will lemmatize on Noun POS and stem the text'''
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='n'))
    #return (WordNetLemmatizer().lemmatize(text,pos='n'))

#tokenize and lemmatize
def preprocess(text):
    '''Function to break into word tokens, remove stopwords, remove short words and finally to lemmatize and stem the individual tokens'''
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

'Hotel: Hotel Arena. Guest Review:  Only the park outside of the hotel was beautiful .  I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next

In [29]:
from langchain_community.document_loaders import TextLoader
raw_documents = TextLoader('used_data.txt').load()
result = preprocess(raw_documents[0].page_content)
print(result)

['hotel', 'hotel', 'arena', 'guest', 'review', 'park', 'outsid', 'hotel', 'beauti', 'angri', 'post', 'avail', 'possibl', 'site', 'plane', 'trip', 'mistak', 'book', 'place', 'book', 'book', 'stay', 'night', 'hotel', 'juli', 'arriv', 'place', 'small', 'room', 'floor', 'hotel', 'turn', 'room', 'book', 'special', 'reserv', 'level', 'duplex', 'room', 'window', 'high', 'ceil', 'room', 'mind', 'broken', 'window', 'close', 'hello', 'rain', 'mini', 'fridg', 'contain', 'sort', 'weapon', 'guess', 'smell', 'intim', 'ask', 'chang', 'room', 'explain', 'time', 'book', 'duplex', 'cost', 'simpl', 'doubl', 'volum', 'high', 'ceil', 'offer', 'room', 'check', 'clock', 'order', 'room', 'wane', 'best', 'begin', 'holiday', 'wait', 'till', 'order', 'check', 'room', 'wonder', 'waist', 'time', 'room', 'want', 'peac', 'intern', 'garden', 'view', 'window', 'tire', 'wait', 'room', 'place', 'belong', 'rush', 'citi', 'even', 'turn', 'constant', 'nois', 'room', 'guess', 'vibrat', 'vent', 'tube', 'constant', 'annoy', '

In [32]:
processed_docs = []

#uncomment below line to find the topics for a particular sentiment
#reviews = reviews[reviews['sentiment']==1]

for doc in Hotel_Reviews['MergedColumn'][:200]:
    processed_docs.append(preprocess(doc))
print(processed_docs[:10])


[['hotel', 'hotel', 'arena', 'guest', 'review', 'park', 'outsid', 'hotel', 'beauti', 'angri', 'post', 'avail', 'possibl', 'site', 'plane', 'trip', 'mistak', 'book', 'place', 'book', 'book', 'stay', 'night', 'hotel', 'juli', 'arriv', 'place', 'small', 'room', 'floor', 'hotel', 'turn', 'room', 'book', 'special', 'reserv', 'level', 'duplex', 'room', 'window', 'high', 'ceil', 'room', 'mind', 'broken', 'window', 'close', 'hello', 'rain', 'mini', 'fridg', 'contain', 'sort', 'weapon', 'guess', 'smell', 'intim', 'ask', 'chang', 'room', 'explain', 'time', 'book', 'duplex', 'cost', 'simpl', 'doubl', 'volum', 'high', 'ceil', 'offer', 'room', 'check', 'clock', 'order', 'room', 'wane', 'best', 'begin', 'holiday', 'wait', 'till', 'order', 'check', 'room', 'wonder', 'waist', 'time', 'room', 'want', 'peac', 'intern', 'garden', 'view', 'window', 'tire', 'wait', 'room', 'place', 'belong', 'rush', 'citi', 'even', 'turn', 'constant', 'nois', 'room', 'guess', 'vibrat', 'vent', 'tube', 'constant', 'annoy', 

In [35]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.keys()[:20]
#print a few words in the dictionary
count = 0
for k,v in dictionary.iteritems():
    print(k,v)
    count = count + 1
    if count > 20:
        break

0 abl
1 angri
2 annoy
3 arena
4 arriv
5 ask
6 asleep
7 attach
8 audio
9 avail
10 beauti
11 begin
12 belong
13 best
14 book
15 broken
16 came
17 caus
18 ceil
19 chang
20 check


In [40]:
dictionary.filter_extremes(no_below=5,no_above=0.1,keep_n=None)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

document_num = 1
bow_doc_x = bow_corpus[1]

for i in range(len(bow_doc_x)):
    print(f'Word {bow_doc_x[i][0]} {dictionary[bow_doc_x[i][0]]} appears {bow_doc_x[i][1]} times')

Word 8 check appears 1 times
Word 20 offer appears 1 times
Word 24 site appears 1 times
Word 37 amsterdam appears 1 times
Word 38 excel appears 1 times
Word 39 fantast appears 1 times
Word 40 origin appears 1 times
Word 41 qualiti appears 1 times
Word 42 real appears 1 times
Word 43 recommend appears 1 times
Word 44 return appears 1 times
Word 45 second appears 1 times
Word 46 servic appears 1 times
Word 47 surround appears 1 times
Word 48 thing appears 1 times
Word 49 thought appears 1 times


In [41]:
seed = 41
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,num_topics=12,id2word=dictionary,passes=10,workers=2,
                                      random_state=seed,minimum_probability=0.05,alpha='symmetric')

In [42]:
'''
For each topic, explore each word and its relative weight in the topic
'''

for idx,topic in lda_model.print_topics(-1,num_words=15):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.052*"servic" + 0.026*"outsid" + 0.025*"design" + 0.025*"coffe" + 0.021*"sleep" + 0.020*"morn" + 0.020*"poor" + 0.019*"star" + 0.017*"upgrad" + 0.017*"nois" + 0.017*"cold" + 0.017*"leav" + 0.017*"think" + 0.017*"complet" + 0.017*"ask"


Topic: 1 
Words: 0.040*"spacious" + 0.035*"narrow" + 0.032*"stair" + 0.028*"posit" + 0.027*"steep" + 0.021*"chang" + 0.021*"quirki" + 0.021*"suit" + 0.021*"expens" + 0.017*"bath" + 0.017*"outsid" + 0.014*"couldn" + 0.014*"citi" + 0.014*"block" + 0.014*"standard"


Topic: 2 
Words: 0.033*"dirti" + 0.033*"euro" + 0.028*"ask" + 0.022*"better" + 0.022*"charg" + 0.022*"came" + 0.017*"price" + 0.017*"glass" + 0.017*"experi" + 0.017*"expect" + 0.017*"told" + 0.017*"gave" + 0.017*"super" + 0.017*"suit" + 0.017*"select"


Topic: 3 
Words: 0.026*"centr" + 0.026*"citi" + 0.025*"feel" + 0.025*"transport" + 0.024*"toilet" + 0.021*"near" + 0.019*"glass" + 0.019*"day" + 0.016*"need" + 0.016*"train" + 0.016*"travel" + 0.016*"modern" + 0.016*"practic" 

In [45]:
#import Coherence model from gensim
from gensim.models import CoherenceModel
#compute coherence score
lda_model_coherence = CoherenceModel(model=lda_model,texts=processed_docs,dictionary=dictionary,
                                    coherence='c_v')
coherence_lda = lda_model_coherence.get_coherence()
print('\nCoherence Score:',coherence_lda)

#define a helper function
def compute_coherence_score(corpus,dictionary,k,a):
    #instantiate the model instance based on k,a and b
    lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=dictionary,num_topics=k,alpha=a,
                                           passes=10,
                                           random_state=seed)
    lda_model_coherence = CoherenceModel(model=lda_model,texts=processed_docs,dictionary=dictionary,coherence='c_v')
    return lda_model_coherence.get_coherence()
#search for the best alpha and the number of topics --> one with the highest coherence score will be the best hyperparameter
alpha =['symmetric','asymmetric']

for x in alpha:
    print('Coherence Model with alpha = {}'.format(x))
    print('-------------------------------------------')
    print('\n')
    for i in range(5,10):
        score = compute_coherence_score(corpus=bow_corpus,dictionary=dictionary,k=i,a=x)        
        print(f'Coherence score with {i} topics is {score}')
        print('\n')


Coherence Score: 0.30617710870419024
Coherence Model with alpha = symmetric
-------------------------------------------


Coherence score with 5 topics is 0.30115071171208185


Coherence score with 6 topics is 0.31703830593030097


Coherence score with 7 topics is 0.3115499452323757


Coherence score with 8 topics is 0.3237309728323424


Coherence score with 9 topics is 0.3453273546027397


Coherence Model with alpha = asymmetric
-------------------------------------------


Coherence score with 5 topics is 0.31423428087240557


Coherence score with 6 topics is 0.2884706212534562


Coherence score with 7 topics is 0.2866913166879253


Coherence score with 8 topics is 0.32692836926736446


Coherence score with 9 topics is 0.32421731812758553




In [46]:
'''
Though the model with alpha='asymmetric' and with 12 topics returned the highest coherence score. For the business use and simplicity, max topics of 7
and alpha='asymmetric' as the final model. This also returned a comparable coherence score of 0.6262
'''
lda_model_final = gensim.models.LdaMulticore(corpus=bow_corpus,num_topics=7,id2word=dictionary,passes=10,workers=2,alpha='asymmetric')

from pprint import pprint
# Print the Keyword in the 7 topics
pprint(lda_model_final.print_topics())
doc_lda = lda_model_final[bow_corpus]

[(0,
  '0.015*"servic" + 0.014*"small" + 0.013*"stair" + 0.013*"manag" + '
  '0.012*"bed" + 0.012*"perfect" + 0.012*"have" + 0.011*"modern" + '
  '0.011*"free" + 0.011*"easi"'),
 (1,
  '0.019*"ticket" + 0.019*"check" + 0.018*"desk" + 0.017*"told" + '
  '0.016*"peopl" + 0.015*"close" + 0.015*"design" + 0.015*"posit" + '
  '0.015*"open" + 0.014*"get"'),
 (2,
  '0.040*"money" + 0.036*"told" + 0.031*"said" + 0.024*"charg" + 0.022*"need" '
  '+ 0.019*"wasn" + 0.018*"extrem" + 0.017*"return" + 0.016*"sleep" + '
  '0.015*"ask"'),
 (3,
  '0.037*"feel" + 0.026*"coffe" + 0.023*"major" + 0.021*"high" + '
  '0.021*"staircas" + 0.019*"glass" + 0.019*"issu" + 0.017*"water" + '
  '0.016*"mention" + 0.016*"drink"'),
 (4,
  '0.032*"outsid" + 0.026*"chang" + 0.023*"check" + 0.021*"window" + '
  '0.021*"nois" + 0.019*"water" + 0.017*"need" + 0.017*"smell" + 0.017*"high" '
  '+ 0.017*"star"'),
 (5,
  '0.054*"recommend" + 0.053*"travel" + 0.053*"euro" + 0.042*"stair" + '
  '0.041*"train" + 0.032*"airport" 